In [5]:
import math
from operator import xor
from functools import reduce
from itertools import compress
from itertools import islice
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

from itertools import islice
from functools import reduce
from operator import xor

def reverse(x, nbit=None):
    ''' reverse the bit order of the input `x` (int) represented with `nbit` 
    (int) bits (e.g., nbit: 5, x: 13=0b01101 -> output: 22=0b10110) '''
    if nbit is None:
        nbit = math.ceil(math.log2(x))
    return int(f'{x:0{nbit}b}'[::-1][:nbit], 2)

def parity(x):
    ''' compute the parity bit of an integer `x` (int) '''
    return bool(sum(int(b) for b in f'{x:b}') % 2)

def int_to_sparse(integer):
    ''' transform an integer into the list of indexes corresponding to 1
    in the binary representation (sparse representation)'''
    sparse = [i for i, b in enumerate(f'{integer:b}'[::-1]) if bool(int(b))]
    return sparse

def sparse_to_int(sparse):
    ''' transform a list of indexes (sparse representation) in an integer whose
    binary representation has 1s at the positions corresponding the indexes and 
    0 elsewhere '''
    integer = sum([2**index for index in sparse])
    return integer

def bits_to_int(bits):
    ''' transform a bit sequence (str of 1/0 or list of bool) into an int '''
    integer = sum(1 << i for i, bit in enumerate(bits) if bool(int(bit)))
    return integer

In [6]:
class LFSR(object):
    '''
    Class implementing a Linear Feedback Shift Register (LFSR)
    
    Attributes
    ----------
    poly: list of int,
        feedback polynomial expressed as list of integers corresponding to
        the degrees of the non-zero coefficients.
    state: int,
        state of the shift register.
    output: bool,
        last shift register output,
    length: int,
        length of the shift register as well as  maximum degree of the feedback
        polynomial.
    
    Methods
    -------
    run_steps(self, N=1)
        Execute multiple LFSR steps
    cycle(self, state=None)
        Execute a full cycle.
    '''
    
    def __init__(self, poly, state=None):
        '''
        Parameters
        ----------
        poly: list of int,
            feedback polynomial expressed as list of integers corresponding to
            the degrees of the non-zero coefficients.
        state: int, optional (default=None)
            shift register initial state.
            If None, state is set to all ones.
        '''
        length = max(poly)
        self._length = length
        self._poly = sparse_to_int(poly) >> 1 # p0 is omitted (always 1)
        
        self._statemask = (1 << length) - 1
        if state is None:
            state = self._statemask
        self.state = state
        
        self._outmask = 1 << (length-1)
        self._output = bool(self._state & self._outmask)
        
        self._feedback = parity(self._state & self._poly) 

    # ==== state ====
    @property
    def state(self):
        # state is re-reversed before being read
        return reverse(self._state, len(self))
    @state.setter
    def state(self, state):
        if not isinstance(state, int):
            raise TypeError('input type is not supported')
        # ensure seed is in the range [1, 2**m-1]
        if (state < 1) or (state > len(self)):
            state = 1 + state % (2**len(self)-2)
        self._state = reverse(state & self._statemask, len(self))

    # ==== length ====
    @property
    def length(self):
        return self._length
    @length.setter
    def length(self, val):
        raise AttributeError('Denied')
    
    # ==== poly ====
    @property
    def poly(self):
        return int_to_sparse((self._poly << 1) | 1)[::-1]
    @poly.setter
    def poly(self, poly):
        raise AttributeError('Denied')

    # ==== output ====
    @property
    def output(self):
        return self._output
    @output.setter
    def output(self, val):
        raise AttributeError('Denied')

    # ==== feedback ====
    @property
    def feedback(self):
        return self._feedback
    @feedback.setter
    def feedback(self, feedback):
        self._feedback = bool(feedback)
    
    
    def __str__(self):
        poly = ' + '.join([
            (f'x^{d}' if d > 1 else ('x' if d==1 else '1')) 
            for d in self.poly
        ])
        string = ', '.join([
            f'poly: "{poly}"',
            f'state: 0x{self.state:0{(self.length+1)//4}x}',
            f'output: {None if self.output is None else int(self.output)}'
        ])
        return string
        
    def __repr__(self):
        return f'LSFR({str(self)})'
    
    def __len__(self):
        return self.length
    
    def __iter__(self):
        return self
    
    def __next__(self):
        '''Execute a LFSR step and returns the output bit (bool)'''
        self._state = ((self._state << 1) | self._feedback) & self._statemask
        self._output = bool(self._state & self._outmask)
        self._feedback = parity(self._state & self._poly) 
        return self._output
    
    def run_steps(self, n=1):
        '''
        Execute multiple LFSR steps.
        
        Parameters
        ----------
        n: int, optional (default=1)
            number of steps to execute.
        
        Output
        ------
        list of bool (len=n),
            LFSR output bit stream.
        '''
        return [next(self) for _ in range(n)]
   
    def cycle(self, state=None):
        '''
        Execute a full LFSR cycle (LFSR.len steps).
        
        Parameters
        ----------
        state: int or list of int or bools, optional (default=None)
            shift register state. If None, state is kept as is.
        
        Output
        ------
        list of bool (len=2**myLFSR.len - 1),
            LFSR output bit stream.
        '''
        if state is not None:
            self.state = state
        return self.run_steps(n=int(2**len(self)) - 1)

poly = [3, 1, 0]
state = 0x7
niter = 7

def print_lfsr(lfsr):
    print(f'{lfsr.state:0{len(lfsr)}b} ({lfsr.state:d}) ',
          f'{int(lfsr.output):d}  {int(lfsr.feedback):d}')

# create and instance of the LFSR
lfsr = LFSR(poly, state)

print('\n state   b fb')
print_lfsr(lfsr) # print initial state
for b in islice(lfsr, niter):
    print_lfsr(lfsr)


 state   b fb
010 (2)  0  0
001 (1)  1  1
100 (4)  0  1
110 (6)  0  1
111 (7)  1  0
011 (3)  1  1
101 (5)  1  0
010 (2)  0  0


In [7]:
class BerlekampMassey():
    '''
    Berlekamp-Massey Algorithm. 
    The algorithm finds the shortest LFSR for a given binary sequence.
    
    This class returns a function whose call method takes one bit at a time as 
    input and returns the feedback polynomial of the shortest LFSR capable of 
    generating the sequence of all bits received up to the last one.
    
    Attributes
    ----------
    poly: list of int,
        feedback polynomial expressed as list of integers corresponding to
        the degrees of the non-zero coefficients.
    
    Methods
    -------
    __call__(bit):
        update and return the polynomial of the shortest LFSR for the observed 
        bit sequence.
    discrepancy():
        compute the discrepancy bit that is 0 (False) if the polynomial `poly` 
        explain the observed bit sequence, or 1 (True) otherwise.
    '''
    
    def __init__(self):
        ''' class constructor '''
        # init algorithm's internal variables
        self.P, self.m = 0x1, 0
        self.Q, self.r = 0x1, 1
        # init an empty sequence of bits
        self.bits = 0 # self.bits = []
        self.tau = 0
    
    # ==== poly ====
    @property
    def poly(self):
        return int_to_sparse(self.P)[::-1]
    @poly.setter
    def poly(self, val):
        raise AttributeError('Denied')

    def discrepancy(self):
        b = self.bits & ((1 << (self.m + 1)) - 1)
        d = parity(self.P & b)
        return d
    
    def __call__(self, bit):
        '''
        Update the feedback polynomial characterizing the shortest LFSR capable 
        of generating the sequence of all bits received.
    
        Parameters
        ----------
        bit: bool, int, or 0/1 str
            input bit
        
        Return
        ------
        poly: list of int,
            feedback polynomial expressed as list of integers corresponding to
            the degrees of the non-zero coefficients.
        '''
        # append
        self.bits = (self.bits << 1) | int(bit)  # self.bits.append(bit)
        if self.discrepancy():
            if 2*self.m <= self.tau: # A                
                self.P, self.Q = self.P ^ (self.Q << self.r), self.P
                self.m, self.r = self.tau + 1 - self.m, 0
                self.bits &= (1<<self.m) - 1  # self.bits = self.bits[-self.m:]
            else: # B
                self.P = self.P ^ (self.Q << self.r)
        self.r += 1
        self.tau += 1
        
        return self.poly

# poly = [3, 1, 0]
poly = [11, 2, 0]

lfsr = LFSR(poly)

berlekamp_massey = BerlekampMassey()
for bit in islice(lfsr, 2**len(lfsr)-1):
    poly = berlekamp_massey(bit)
poly

[11, 2, 0]

In [8]:
class PseudoRandomByteGenerator():
    '''
    Class implementing a pseudo random byte generator starting from a pseudo
    random bit generator.
    '''
    def __init__(self, seed=None, bit_generator=None, **kwargs):
        '''
        Parameters
        ----------
        seed: int or bytes, optional (default None)
            Seed for the pseudo random bit generator.
            If None, the value 0 is adopted
        bit_generator: iterator, optional (default None)
            Pseudo random bit generator. It must must takes seed (int) as first 
            positional arguments and return an iterable that yields a bit.
            If None, an LFSR with feedback polynomial x^12+x^6+x^4+x+1 is 
            adopted.
        kwargs: mapping, optional
            a dictionary of keyword arguments passed into `bit_generator`.
        '''
        if seed is None:
            seed = 0
        if isinstance(seed, bytes):
            seed = int.from_bytes(seed, byteorder='little')
        if not isinstance(seed, int):
            raise TypeError(f'Unsupported type {type(seed)} for seed')
            
        if bit_generator is None:
            poly = [12, 6, 4, 1, 0]
            if seed == 0:
                seed = None
            self.bit_generator = LFSR(poly, state=seed)
        else:
            self.bit_generator = bit_generator(seed, **kwargs)
    
    def __iter__(self):
        return self
    
    def __next__(self):
        '''Generate a Pseudo Random Byte (int)'''
        byte = bits_to_int([bit for bit in islice(self.bit_generator, 8)])
        return byte

In [9]:
class StreamCipher(object):
    '''
    Stream cipher.

    Methods
    -------
    encrypt(self, plaintext):
        encrypt a plaintext
    decrypt(self, ciphertext): 
        decrypt a ciphertext
    '''
    def __init__(self, key, prng=None, **kwargs):        
        '''
        Parameters
        ----------
        key: int or bytes,
            secret key for PRNG initialization
        prng: iterator, optional (default None),
            pseudo random number generator (PRNG) for the generation of
            the random byte used for encryption and decryption
        kwargs: dict,
            keyword arguments for `prng`
        '''
        if prng is None:
            self.prng = PseudoRandomByteGenerator(key)
        else:
            self.prng = prng(key, **kwargs)
        
    def encrypt(self, plaintext):
        '''encrypt a `plaintext` (str, bytes) and  return the corresponding 
        cyphertext (bytes) '''
        return self._crypt(plaintext)
    
    def decrypt(self, ciphertext):
        ''' decrypt a `cypertext` (str, bytes) and return the corresponding 
        plaintext (bytes) '''
        return self._crypt(ciphertext)
    
    def _crypt(self, text):
        if isinstance(text, str):
            text = text.encode('utf-8')
        crypted = bytes([b^s for b, s in zip(text, self.prng)])
        return crypted
        

In [10]:
message = 'hello world!'
key = 0x0123456789ABCDEF

# create a StreamCipher instance for Alice and Bob
alice = StreamCipher(key) 
bob   = StreamCipher(key)

plaintextA = message.encode('utf-8')   # string to bytes 
ciphertext = alice.encrypt(plaintextA) # encryption by Alice
plaintextB = bob.decrypt(ciphertext)   # decryption by Bob

print(plaintextA) # -> b'hello world!' 
print(ciphertext) # -> b'U\xfe7\xaa\xc8\n\xc9\xa2\x81v|\xc3'
print(plaintextB) # -> b'hello world!' 

b'hello world!'
b'U\xfe7\xaa\xc8\n\xc9\xa2\x81v|\xc3'
b'hello world!'


In [26]:
from bitarray import bitarray
import binascii 
import codecs

def access_bit(data, num):
    base = int(num // 8)
    shift = int(num % 8)
    return (data[base] & (1<<shift)) >> shift

with open("proj1_ciphertext.bin", "rb") as ciphertext:
    cipher_data = ciphertext.read()
    #print(cipher_data)
    cipher_bitstream = [access_bit(cipher_data,i) for i in range(len(cipher_data)*8)]
    #print(cipher_bitstream)
    #bitstream = ''.join(bits)

with open("proj1_known-plaintext.txt", 'r') as plaintext:
    ba = bitarray()
    plain_data = plaintext.read()
    plain_bitstream = bin(int(binascii.hexlify(bytearray(plain_data, 'utf-8')),16))
    plain_bits = [int(plain_bitstream[i]) for i in range(2, len(plain_bitstream))]
    #print(plain_bits)

    """
    bytearrayy = bytearray(data, 'utf-8')
    print(bytearrayy)

    plain_bits = [format(x, 'b') for x in bytearray(data, 'utf-8')]
    print(plain_bits)
    plain_bitstream = [int(i) for i in plain_bits]
    print(plain_bitstream)
"""
    
def xor_texts():   
    key = []
    print(type(plain_bitstream[0]))
    print(type(cipher_bitstream[0]))
    for i in range(len(plain_bits)): 
        #print(xor(plain_bits[i], cipher_bitstream[i]))
        key.append(xor(plain_bits[i], cipher_bitstream[i]))
    return_key = [int(i) for i in key]
    #print(return_key)
    return return_key

keyy = xor_texts()  

berlekamp_massey = BerlekampMassey()
for bit in keyy:
    poly = berlekamp_massey(bit)
k = 10101110010110100010111001011111110001110110001001110111101110110100100111100000000100101000010001111110011110101011101100101011011111101110011001001010111000101001101101110001011100010010110000010000000110100100010111100111100100100001010001110001101110010001100001100110111111001111000110101100010110000110011011100101000010111010011011101001010001100011101011001001000001010010000101100101100000100101001100001000011111000111111111010000101111001010001011010001001100000011101101000010010100011111001110111110000100111000110111101100110001000100100001111100011001101010100011110010001110111011010101101101101111011110000101110111010100101010110001001111010001101011000100110000011101100011100101001110000001010101001111111000101001010111000000110101110001011011000110110101111111110011110010011000001100000011010110011000010010110111100101100110100111100100011110110110110100100011110001101000101011011101010000010001100110001000010000101010111110110110100000101000111101110011110001100110000110111010001001110101100001100100110110100100110011011111110000110000100011111101111111110100010100101100011

ke = [str(i) for i in keyy]
key = ''.join(ke)
bob = StreamCipher(key=k, poly=poly)

#plaintextA = plain_data.encode('utf-8')
#ciphertext = streamcipher.encrypt(plaintextA)
#a = streamcipher.encrypt(ciphertext)
"""
alice = StreamCipher(k, poly=poly)
plaintextA = plain_data.encode('utf-8')
ciphertext = alice.encrypt(plaintextA)
bob = StreamCipher(key, poly=poly)
plaintextB = bob.encrypt(ciphertext)
"""
"""
print('Plaintext', plaintextA)
print('Ciphertext: ', ciphertext)
print('Plain text: ', plaintextB)
"""

plaintextB = bob.encrypt(cipher_data)[:15]
#answer = bytearray.fromhex(decoded).decode()
answer = plaintextB.decode('cp1252')
print(answer)

<class 'str'>
<class 'int'>
÷ÂÐìþ(
?	¾
